In [1]:
import PyQt5
%matplotlib qt
import pyzed.sl as sl
from ZED import ZED
from TRACKING import handtracking
import cv2
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
# Run this Block to Run Hand Tracking with LIVE STREAM VIDEO
zed = ZED(fps=60)

In [2]:
# Run this Block to Run Hand Tracking with RECORDED VIDEO (.SVO FILE)
# And Specify .svo File Name
input_name = 'test.svo'
zed = ZED(fps=60, input_filename = input_name)

Reading SVO File: 0


In [3]:
zed.config_resolution("HD720")
zed.config_depth_mode("NEURAL")
zed.open_camera(fill_mode=True)
zed.get_calibration_params()
fx = zed.fx
fy = zed.fy
cx = zed.cx
cy = zed.cy
width = zed.camera.get_camera_information().camera_configuration.resolution.width
height = zed.camera.get_camera_information().camera_configuration.resolution.height

Open ZED Successfully!


In [4]:
detector = handtracking()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [ ]:
fig = plt.figure()
plt.ion()
ax = fig.add_subplot(111, projection='3d')

In [ ]:
while True:
    zed.camera.grab(zed.runtime_params)
    zed.get_image()
    image = zed.image_cv2
    point_cloud = zed.point_cloud
    image = detector.detect_landmarks(image)
    data_left, data_right = detector.get_position(image, point_cloud, fx = fx, fy = fy, width=width, height=height)

    if data_right.shape == (21,3) and data_left.shape == (21,3):
        data_plot = np.vstack((data_right,data_left))
    elif data_right.shape == (21,3):
        data_plot = data_right
    elif data_left.shape == (21,3):
        data_plot = data_left
    else:
        data_plot = np.array([])
    detector.plot(ax,plt,data_plot)

    cv2.imshow("test", image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()